In [ ]:
%pip install numpy

In [ ]:
%pip install sentencepiece

In [ ]:
import numpy as np


In [ ]:
import sentencepiece as spm

Tokenization Layer

In [ ]:
sp=spm.SentencePieceProcessor(model_file = 'spm_unigram_en.model')

In [ ]:
def tokenize_text(input_text):
    ids = sp.encode_as_ids(text)
    return ids


_IncompleteInputError: incomplete input (2089606159.py, line 1)

In [ ]:
model_dim = 512
vocab_size = sp.get_piece_size()
def token_embedding( token_ids):
    embedding_matrix = np.random.radn(vocab_size, model_dim)
    embeddings = embedding_matrix[token_ids]
    return embeddings

_IncompleteInputError: incomplete input (2574531015.py, line 3)

Positional encoding matrix

In [ ]:
def positional_encoding(max_sequence_length , dmodel):
    positional_encoding_matrix = np.zeros((max_sequence_length , dmodel))
    for position in range(max_sequence_length):
        for i in range(0,dmodel//2):
            positional_encoding_matrix[position , 2*i] = np.sin(position/(10000**(2*i/dmodel)))
            positional_encoding_matrix[position , 2*i +1]=  np.cos(position/(10000**(2*i/dmodel)))
    return positional_encoding_matrix
                                                                              


Scaled Dot product Attention

In [ ]:
def ScaledDotProductAttetnion(Q,K,V):
    scaled_scores = (np.matmul(Q,K.T))/np.sqrt(Q.shape[-1])
    exp_scores = np.exp(scaled_scores - np.max(scaled_scores, axis=-1,keepdims=True)) #subtract the max for numerical stability(Prevents np.exp() from overflowing when numbers are large)
    attention_weights = exp_scores / np.sum(exp_scores, axis=-1 ,keepdims=True)
    attention_score = np.matmul(attention_weights ,V)
    return attention_score



In [27]:
Q = np.array([[2,3,4],
               [5,5,6]])

K = np.array([[2,35,5],
               [5,5,4]])

V = np.array([[2,3,2],
               [5,4,5]])
attn_score = ScaledDotProductAttetnion(Q,K,V)
print(attn_score)

[[2. 3. 2.]
 [2. 3. 2.]]


In [36]:

arr = np.array([[[1, 2, 3],
                 [4, 5, 6]],

                [[7, 8, 9],
                 [10, 11, 12]]])
print(arr.shape)  
print(arr.transpose(0,2,1))

(2, 2, 3)
[[[ 1  4]
  [ 2  5]
  [ 3  6]]

 [[ 7 10]
  [ 8 11]
  [ 9 12]]]


Tensor Surgery(Splitting Heads ,reshaping , transposing , Combining heads )

In [48]:
def split_heads(tensor , batch_size , heads):
    dimension_head = int(tensor.shape[-1]/ heads) #shape[-1] takes the dimension of tensor 
    sequence_length = tensor.shape[1]
    x = tensor.reshape(batch_size , sequence_length , heads , dimension_head)
    final_tensor = x.transpose(0 , 2 , 1 , 3)
    return final_tensor


In [50]:
B , L , D = 32 , 10 , 512
H=8
example_tensor = np.zeros(( B , H , D )) # np.zeros mocks data
reshaped_tensor= split_heads(example_tensor,B,H )
print(f"Input Shape :{example_tensor.shape}")
print(f"Output  Shape :{reshaped_tensor.shape}")


Input Shape :(32, 8, 512)
Output  Shape :(32, 8, 8, 64)


In [ ]:
def combine_heads(tensor):
    batch_size = tensor.shape[0]
    heads = tensor.shape[1]
    sequence_length = tensor.shape[2]
    dimension_head = tensor.shape[3]
    x = tensor.transpose(0 , 2 , 1 , 3)
    combined_tensor = x.reshape(batch_size , sequence_length , heads * dimension_head)
    return combined_tensor

Sub-layers components

In [ ]:
def residual_connection( x , sublayer_output_FNN_or_attention):
    return x + sublayer_output

def layer_normalization (residual_output ,gamma , beta , epsilon = 1e-5 ): #here gamma and beta are learnable parameters (weights and bias)
    mean = np.mean( residual_output , axis = -1 , keepdims = True)
    st_deviation= np.std( residual_output , axis = -1 , keepdims = True)
    normalized_output = ((residual_output -mean)/(st_deviation+epsilon) )*gamma + beta
    return normalized_output


The Feed-Forward Network(FFN)(In tranformer paper , FFN contains two linear layer + RELU activation function)

In [ ]:
def feed_forward(output_from_attention , W1 , B1 , W2 , B2):
    linear_transform1 = np.matmul(output_from_attention , W1) + B1
    activation_output = np.maximum (0 , linear_transform1) #ReLU activation
    linear_transform2 = np.matmul(activation_output , W2) + B2
    return linear_transform2


Initializing weights

In [ ]:
def initialize_parameters(dmodel , heads , d_ff):
    dimension_head = dmodel // heads
    parameters = {}
    parameters['encodding_matrix'] = 
    parameters['Wq'] = np.random.randn(dmodel , dmodel) / np.sqrt(dmodel)
    parameters['Wk'] = np.random.randn(dmodel , dmodel) / np.sqrt(dmodel)
    parameters['Wv'] = np.random.randn(dmodel , dmodel) / np.sqrt(dmodel)
    parameters['Wo'] = np.random.randn(dmodel , dmodel) / np.sqrt(dmodel)
    parameters['W1'] = np.random.randn(dmodel , d_ff) / np.sqrt(dmodel)
    parameters['B1'] = np.zeros((1 , d_ff))
    parameters['W2'] = np.random.randn(d_ff , dmodel) / np.sqrt(d_ff)
    parameters['B2'] = np.zeros((1 , dmodel))
    parameters['gamma1'] = np.ones((1 , dmodel))
    parameters['beta1'] = np.zeros((1 , dmodel))
    parameters['gamma2'] = np.ones((1 , dmodel))
    parameters['beta2'] = np.zeros((1 , dmodel))
    return parameters
    
    

Encoder Layer

In [ ]:
def encoder_layer(input_tensor , parameters , heads):
    Wq , Wk , Wv , Wo = parameters['Wq'] , parameters['Wk'] , parameters['Wv'] , parameters['Wo']
    W1 , B1 , W2 , B2 = parameters['W1'] , parameters['B1'] , parameters['W2'] , parameters['B2']
    gamma1 , beta1 , gamma2 , beta2 = parameters['gamma1'] , parameters['beta1'] , parameters['gamma2'] , parameters['beta2']
    Q , K , V = np.matmul(input_tensor , Wq) , np.matmul(input_tensor , Wk) , np.matmul(input_tensor , Wv)
    batch_size = input_tensor.shape[0]
    Q_heads , K_heads , V_heads = split_heads(Q , batch_size , heads)   , split_heads(K , batch_size , heads) , split_heads(V , batch_size , heads)
    attention_output_heads = ScaledDotProductAttetnion(Q_heads , K_heads , V_heads)
    combined_attention_output = combine_heads(attention_output_heads)
    attention_output = np.matmul(combined_attention_output , Wo)
    residual_connection1 = residual_connection(input_tensor , attention_output)
    normalized_output1 = layer_normalization(residual_connection1 , gamma1 , beta1)
    ffn_output = feed_forward(normalized_output1 , W1 , B1 , W2 , B2)
    residual_connection2 = residual_connection(normalized_output1 , ffn_output)
    normalized_output2 = layer_normalization(residual_connection2 , gamma2 , beta2)
    return normalized_output2
    

    

Decoder Layer